In [40]:
def get_info():
    # Import Libraries
    import requests, json, os, sys, time
    import pandas as pd
    from datetime import datetime

    '''
    Preparing folder variables.
    '''
    os.chdir(os.path.dirname(sys.path[0])) # This command makes the notebook the main path and can work in cascade.
    main_folder = sys.path[0]
    data_folder = (main_folder + "\data")

    '''
    Creating time variables.
    '''
    current_time = time.strftime("%H_%M_%S",time.localtime())
    date = datetime.now()
    actual_date = date.strftime("%Y_%m_%d")


    #Remove the limit to see the df
    pd.set_option('display.max_columns', None)

    #Creating the necessary lists
    anime_list = []

    '''
    To check if there is an empty value. If the category is empty, it returns None.
    '''
    def try_it(i):
        try:
            return i["name"]
        except:
            return None

    '''
    Def with a try check to get the finishing time of an anime, in case the anime is a movie, then it returns the release time.
    If the anime is not a movie, it checks for the finishing time. If there is no finishing time, it returns None 
    '''

    url = "https://api.jikan.moe/v4/top/anime" # url of the api

    r = requests.get(url)# request to a web page (url)    
    
    data = r.json() # creating a variable for all the info we get

    n_pages = data['pagination']['last_visible_page']
    
    for page in range (1,n_pages +1):
        r_page = requests.get(url + '?page=' + str(page)) # request to a web page (url)
        content = r_page.json()
        print (page)
        data = content["data"]
        time.sleep(1)
        for char in data: 
            
            try: # First try yo check if the page exist or not
                # Creation of the necessary dictionary o store the values in each loop # We specify which information to get in each Item
                anime_dict = {"anime_id" : char["mal_id"] if char["mal_id"]  else None,
                            "Cover" : char["images"]["jpg"]["large_image_url"] if char["images"]["jpg"]["large_image_url"]  else None,
                            "Regular_Title" : char["title_english"] if char["title_english"]  else None,
                            "English_Title" : char["title"] if char["title"]  else None,
                            "Japanses_Title" : char["title_japanese"] if char["title_japanese"]  else None,
                            "Type" : char["type"] if char["type"]  else None,
                            "Source" : char["source"] if char["status"] else None,
                            "Audience" : [try_it(i) for i in char["demographics"]], # List comprehension calling the Def try_it
                            "N_Episodes" : (int(char["episodes"])) if char["episodes"] else 0,
                            "Duration" : char["duration"] if char["duration"]  else None,
                            "Status" : char["status"] if char["status"]  else None,
                            "Rating" : char["rating"] if char["rating"] else None,
                            "Score" : char["score"] if char["score"]  else None,
                            "Scored_by" : char["scored_by"] if char["scored_by"]  else None,
                            "Rank" : (int(char["rank"])) if char["rank"] else None,
                            "Season" : char["season"] if char["season"] else None,
                            "Genre" : [try_it(i) for i in char["genres"]],# List comprehension calling the Def try_it
                            "Theme" : [try_it(i) for i in char["themes"]],# List comprehension calling the Def try_it
                            "Released" : (int(char["aired"]["prop"]["from"]["year"])) if char["aired"]["prop"]["from"]["year"] else None, # If else in one line
                            "Studios" : [try_it(i) for i in char["studios"]],# List comprehension calling the Def try_it
                            "Producers" : [try_it(i) for i in char["producers"]],
                            "synopsis" : char["synopsis"] if char["synopsis"]  else None,# List comprehension calling the Def try_it
                            }
                            
                anime_list.append(anime_dict) # Append the loop info to anime_list
                
            # Ending of the first try specifying the error
            except:
                if r_page.status_code == 429: #If there is a 429 error we show it on screen and tell us the respuesta.reason
                    print (f"El código de estado de la petición es: {r_page.status_code}. Estatus {r_page.reason}. No se puede recoger información de la página {id}\n")
                else:
                    #If there is a any other error we show it on screen and tell us the respuesta.reason
                    print (f"El código de estado de la petición es: {r_page.status_code}. Estatus {r_page.reason}. No se puede recoger información de la página {id}\n")
                continue
            

    # We create df from anime_list and save it in a csv file adding actual date and time variables to the name
    anime_df = pd.DataFrame(anime_list)
    anime_csv = os.path.join(data_folder,"anime_" + actual_date+ "_" +current_time + ".csv")# Saving the image to the images folder
    anime_df.to_csv(anime_csv, sep = ';', index = False)
    print(f'anime_{actual_date}{current_time}.csv created\n\n')

In [ ]:
get_info()

In [1]:
# Standard library imports
import os # allows access to OS-dependent functionalities
import re #  regular expression matching operations similar to those found in Perl
import sys # to manipulate different parts of the Python runtime environment
import warnings # is used to display the message Warning
import pickle # serializing and deserializing a Python object structure.
from utils import cleaning
import numpy as np # functions for working in domain of linear algebra, fourier transform, matrices and arrays
import pandas as pd # data analysis and manipulation tool
import joblib # set of tools to provide lightweight pipelining in Python

#Preparing folder variables
os.chdir(os.path.dirname(sys.path[0])) # This command makes the notebook the main path and can work in cascade.
main_folder = sys.path[0]
data_folder = (main_folder + "\data")
saved_models_folder = (data_folder + "\saved_models")
raw_data = (data_folder + "\_raw")
processed_data = (data_folder + "\processed")
content_based_supervised_data = (data_folder + "\processed\content_based_supervised")

c:\Users\christiandda\miniconda3\envs\stlit\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
df = cleaning.final_df()
df.columns

Index(['anime_id', 'name', 'english_title', 'japanses_title', 'genre', 'type',
       'source', 'duration', 'episodes', 'rating', 'score', 'rank', 'members',
       'synopsis'],
      dtype='object')

In [ ]:
df = df[['anime_id',"name", 'English_Title', 'Japanses_Title',"genre", 'type', 'Source', 'Duration', 'episodes', 'Rating', 'Score',"Rank", 'members', 'synopsis']]

In [13]:
df.columns= df.columns.str.lower()


AttributeError: 'Index' object has no attribute 'columns'

In [14]:
df= df.rename(columns=str.lower)
df.head()

TypeError: Index.rename() got an unexpected keyword argument 'columns'

In [ ]:
anime_final = anime_final[['anime_id', 'english_title', 'japanses_title', 'type', '3']]

In [12]:
anime_final.head()

,english_title,source,duration,rating,rank,anime_id,cover,japanses_title,name,genre,type,episodes,rating,members
0,Cowboy Bebop,Original,24,R - 17+ (violence & profanity),39,1,https://cdn.myanimelist.net/images/anime/4/196...,カウボーイビバップ,cowboy bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26.0,8.82,486824.0
1,Cowboy Bebop: Tengoku no Tobira,Original,115,R - 17+ (violence & profanity),185,5,https://cdn.myanimelist.net/images/anime/1439/...,カウボーイビバップ 天国の扉,cowboy bebop tengoku no tobira,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1.0,8.40,137636.0
2,Trigun,Manga,24,PG-13 - Teens 13 or older,323,6,https://cdn.myanimelist.net/images/anime/7/203...,トライガン,trigun,"Action, Comedy, Sci-Fi",TV,26.0,8.32,283069.0
3,Witch Hunter Robin,Original,25,PG-13 - Teens 13 or older,2766,7,https://cdn.myanimelist.net/images/anime/10/19...,Witch Hunter ROBIN (ウイッチハンターロビン),witch hunter robin,"Action, Drama, Magic, Mystery, Police, Superna...",TV,26.0,7.36,64905.0
4,Bouken Ou Beet,Manga,23,PG - Children,4188,8,https://cdn.myanimelist.net/images/anime/7/215...,冒険王ビィト,beet the vandel buster,"Adventure, Fantasy, Shounen, Supernatural",TV,52.0,7.06,9848.0


In [14]:
anime_final = pd.read_csv(processed_data + "/" + "anime_final.csv")# load anime df
anime_final.head()


,English_Title,Type,Source,N_Episodes,Duration,Rating,Score,Rank,Genre,anime_id,Cover,Japanses_Title,name,genre,type,episodes,rating,members
0,Cowboy Bebop,TV,Original,26,24,R - 17+ (violence & profanity),8.75,39,"Action,Sci-Fi",1,https://cdn.myanimelist.net/images/anime/4/196...,カウボーイビバップ,cowboy bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26.0,8.82,486824.0
1,Cowboy Bebop: Tengoku no Tobira,Movie,Original,1,115,R - 17+ (violence & profanity),8.38,185,"Action,Sci-Fi",5,https://cdn.myanimelist.net/images/anime/1439/...,カウボーイビバップ 天国の扉,cowboy bebop tengoku no tobira,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1.0,8.40,137636.0
2,Trigun,TV,Manga,26,24,PG-13 - Teens 13 or older,8.22,323,"Action,Adventure,Comedy,Drama,Sci-Fi",6,https://cdn.myanimelist.net/images/anime/7/203...,トライガン,trigun,"Action, Comedy, Sci-Fi",TV,26.0,8.32,283069.0
3,Witch Hunter Robin,TV,Original,26,25,PG-13 - Teens 13 or older,7.25,2766,"Action,Drama,Mystery,Supernatural",7,https://cdn.myanimelist.net/images/anime/10/19...,Witch Hunter ROBIN (ウイッチハンターロビン),witch hunter robin,"Action, Drama, Magic, Mystery, Police, Superna...",TV,26.0,7.36,64905.0
4,Bouken Ou Beet,TV,Manga,52,23,PG - Children,6.95,4188,"Adventure,Fantasy,Supernatural",8,https://cdn.myanimelist.net/images/anime/7/215...,冒険王ビィト,beet the vandel buster,"Adventure, Fantasy, Shounen, Supernatural",TV,52.0,7.06,9848.0


In [ ]:
df.rename(columns={'rating': 'score'}, inplace=True)

In [ ]:
anime_final = anime_final.drop(['Genre', 'Type', 'N_Episodes', 'Score',"members"], axis=1, inplace=True)


In [ ]:
anime_final = anime_final.columns= anime_final.columns.str.lower()

In [11]:
cols = list(anime_final.columns.values)
cols

['english_title',
 'source',
 'duration',
 'rating',
 'rank',
 'anime_id',
 'cover',
 'japanses_title',
 'name',
 'genre',
 'type',
 'episodes',
 'rating',
 'members']

In [5]:
anime_final.drop(['Genre', 'Type', 'N_Episodes', 'Score',"members"], axis=1, inplace=True)

In [ ]:
df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'}, inplace=True)

In [6]:
anime_final.head()

,English_Title,Source,Duration,Rating,Rank,anime_id,Cover,Japanses_Title,name,genre,type,episodes,rating,members
0,Cowboy Bebop,Original,24,R - 17+ (violence & profanity),39,1,https://cdn.myanimelist.net/images/anime/4/196...,カウボーイビバップ,cowboy bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26.0,8.82,486824.0
1,Cowboy Bebop: Tengoku no Tobira,Original,115,R - 17+ (violence & profanity),185,5,https://cdn.myanimelist.net/images/anime/1439/...,カウボーイビバップ 天国の扉,cowboy bebop tengoku no tobira,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1.0,8.40,137636.0
2,Trigun,Manga,24,PG-13 - Teens 13 or older,323,6,https://cdn.myanimelist.net/images/anime/7/203...,トライガン,trigun,"Action, Comedy, Sci-Fi",TV,26.0,8.32,283069.0
3,Witch Hunter Robin,Original,25,PG-13 - Teens 13 or older,2766,7,https://cdn.myanimelist.net/images/anime/10/19...,Witch Hunter ROBIN (ウイッチハンターロビン),witch hunter robin,"Action, Drama, Magic, Mystery, Police, Superna...",TV,26.0,7.36,64905.0
4,Bouken Ou Beet,Manga,23,PG - Children,4188,8,https://cdn.myanimelist.net/images/anime/7/215...,冒険王ビィト,beet the vandel buster,"Adventure, Fantasy, Shounen, Supernatural",TV,52.0,7.06,9848.0


In [35]:
anime_final = pd.read_csv(processed_data + "/" + "anime_final.csv")# load anime df
anime_final.head(2)

,English_Title,Type,Source,N_Episodes,Duration,Rating,Score,Rank,Genre,anime_id,Cover,Japanses_Title,name,genre,type,episodes,rating,members
0,Cowboy Bebop,TV,Original,26,24,R - 17+ (violence & profanity),8.75,39,"Action,Sci-Fi",1,https://cdn.myanimelist.net/images/anime/4/196...,カウボーイビバップ,cowboy bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26.0,8.82,486824.0
1,Cowboy Bebop: Tengoku no Tobira,Movie,Original,1,115,R - 17+ (violence & profanity),8.38,185,"Action,Sci-Fi",5,https://cdn.myanimelist.net/images/anime/1439/...,カウボーイビバップ 天国の扉,cowboy bebop tengoku no tobira,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1.0,8.40,137636.0


In [60]:
#The name of the .csv file should be changed for an updated one or use the pickle file
anime_no_cleaned = pd.read_csv(raw_data + "/" + "anime.csv")# load anime df
anime_no_cleaned.head(40)

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855
7,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679
8,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,1,9.10,72534
9,15417,Gintama&#039;: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",TV,13,9.11,81109


In [42]:
#The name of the .csv file should be changed for an updated one or use the pickle file
anime_new = pd.read_csv(raw_data + "/" + "anime_2023_02_15_00_08_28.csv",sep=";")# load anime df
anime_new.head(2)

,anime_id,Cover,English_Title,Japanses_Title,Type,Source,Audience,N_Episodes,Duration,Rating,Score,Scored_by,Rank,Season,Genre,Theme,Released,Studios,Producers,synopsis
0,1,https://cdn.myanimelist.net/images/anime/4/196...,Cowboy Bebop,カウボーイビバップ,TV,Original,[],26,24 min per ep,R - 17+ (violence & profanity),8.75,888674.0,40.0,spring,"['Action', 'Award Winning', 'Sci-Fi']","['Adult Cast', 'Space']",1998.0,['Sunrise'],['Bandai Visual'],"Crime is timeless. By the year 2071, humanity ..."
1,5,https://cdn.myanimelist.net/images/anime/1439/...,Cowboy Bebop: Tengoku no Tobira,カウボーイビバップ 天国の扉,Movie,Original,[],1,1 hr 55 min,R - 17+ (violence & profanity),8.38,201890.0,185.0,NaN,"['Action', 'Sci-Fi']","['Adult Cast', 'Space']",2001.0,['Bones'],"['Sunrise', 'Bandai Visual']","Another day, another bounty—such is the life o..."


In [62]:
import pandas as pd 

anime_merged = pd.merge(anime_new[["anime_id",'English_Title',"Japanses_Title","Source","Duration","Rating","Score","Rank","synopsis"]]\
    , anime_no_cleaned[["anime_id",'name','genre',"type","episodes","members"]]\
    , on = "anime_id")
anime_merged.shape

(12201, 14)

In [64]:
anime_merged.head()

,anime_id,English_Title,Japanses_Title,Source,Duration,Rating,Score,Rank,synopsis,name,genre,type,episodes,members
0,1,Cowboy Bebop,カウボーイビバップ,Original,24 min per ep,R - 17+ (violence & profanity),8.75,40.0,"Crime is timeless. By the year 2071, humanity ...",Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,486824
1,5,Cowboy Bebop: Tengoku no Tobira,カウボーイビバップ 天国の扉,Original,1 hr 55 min,R - 17+ (violence & profanity),8.38,185.0,"Another day, another bounty—such is the life o...",Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1,137636
2,6,Trigun,トライガン,Manga,24 min per ep,PG-13 - Teens 13 or older,8.22,315.0,"Vash the Stampede is the man with a $$60,000,0...",Trigun,"Action, Comedy, Sci-Fi",TV,26,283069
3,7,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),Original,25 min per ep,PG-13 - Teens 13 or older,7.25,2791.0,Robin Sena is a powerful craft user drafted in...,Witch Hunter Robin,"Action, Drama, Magic, Mystery, Police, Superna...",TV,26,64905
4,8,Bouken Ou Beet,冒険王ビィト,Manga,23 min per ep,PG - Children,6.94,4310.0,It is the dark century and the people are suff...,Beet the Vandel Buster,"Adventure, Fantasy, Shounen, Supernatural",TV,52,9848
